In [1]:
import os
os.chdir('../../src/models')

import pandas as pd
import datetime
import pickle as pkl

import plotly.express as px
import plotly.graph_objects as go

from policies import costs

set3 = px.colors.qualitative.Set3

color_map = {
    'Lockdown':          'rgb(0.83, 0.13, 0.15)',
    'Hard Quarantine':    'rgb(0.85, 0.35, 0.13)',
    'Light Quarantine':   'rgb(0.97, 0.91, 0.56)',
    'Social Distancing':  'rgb(0.67, 0.88, 0.69)',
    'Unrestricted':        'rgb(0.86, 0.86, 0.86)'    
}

In [2]:
def make_SIR_graph(data, ):
    d_counts = pd.DataFrame([pd.Series(d).value_counts() for d in data])
    d_counts.fillna(0, inplace=True)

    fig = go.Figure()
    x = list(range(len(d_counts[0])+1))

    fig.add_trace(go.Scatter(x=x, y=d_counts[-1]/55e3, name='removed',line_color='green'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[0]/55e3, name='susceptible', line_color='blue'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[1]/55e3, name='exposed',line_color ='orange'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[2]/55e3, name='infected', line_color = 'red'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[3]/55e3, name='hospitalized', line_color = 'purple'))
    fig.update_layout(hovermode='x')
    fig.show()
    
    return d_counts

def make_beds_graph(data, actions, step_size, title, color_map, make_df = True):
    fig = go.Figure()
    
    if make_df:
        data = pd.DataFrame([pd.Series(d).value_counts() for d in data])
        data.fillna(0, inplace=True)
    
    actions = list(map(color_map.get,  actions))

    x = list(range(len(data)+1))
    
    #\definecolor{royalblue(web)}{rgb}{0.25, 0.41, 0.88}
    fig.add_trace(go.Scatter(x=x, y=data[3]/55e3, name='hospitalized', line_color = 'rgb(0.25, 0.41, 0.88)',
                            line=dict(width=3.5),  yaxis="y2"))
    #\definecolor{firebrick}{rgb}{0.7, 0.13, 0.13}
    fig.add_trace(go.Scatter(x=x, y=data[1]/55e3, name='exposed', line_color = 'rgb(0.7, 0.13, 0.13)',
                            line=dict(width=4)))
    fig.add_trace(go.Scatter(x=x, y=len(data)*[0.0015], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2), yaxis="y2"))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)],
                y1=1,
                fillcolor=a,
                opacity=0.45,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(coloraxis = {'colorscale':'deep'}, xaxis={'showgrid': False,},
                      yaxis = {'showgrid': False, 'zerolinecolor': 'black',
                               'zerolinewidth': .5, 'zeroline': False,  'title': 'Proportion of Population'},
                      showlegend=True, hovermode="x")
    
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = list(range(0, len(data)+1, 14)),
            ticktext = list(range(0, int(len(data)/7) + 1, 2)),
            title = 'Weeks'
            )
        )
    fig.update_layout(showlegend=False)
    
    fig.update_layout(
        yaxis=dict(
            title="Proportion of Population Exposed",
            titlefont=dict(
                color="rgb(0.7, 0.13, 0.13)"
            ),
            showgrid=False,
            tickfont=dict(
                color="rgb(0.7, 0.13, 0.13)"
            ),
            range=[0, .0175]
        ),
        yaxis2=dict(
            title="Proportion of Population Hospitalized",
            titlefont=dict(
                color="rgb(0.25, 0.41, 0.88)"
            ),
            tickfont=dict(
                color="rgb(0.25, 0.41, 0.88)"
            ),
            showgrid=False,
            anchor="x",
            overlaying="y",
            side="right",
            range=[0, .00175]

        ),)

    fig.write_image(f"{title}.pdf")


    fig.show()

In [10]:
files = ['new_MCTS_50_6_280_higherpr1.pkl',
        'new_MCTS_50_6_280_higherpr2.pkl',
        'new_MCTS_50_6_280_higherpr3.pkl',
        'new_MCTS_50_6_280_higherpr4.pkl']
results = []

for file in files:
    with open(file, 'rb') as f:
        results.append(pkl.load(f))

for i,r in enumerate(results):
    df = make_SIR_graph(r[0])
    #actions_colors = list(map(color_map_set3.get,  r[1]))
    make_beds_graph(r[0], r[1], 7, files[i], color_map)
    print(pd.Series(map(costs.get,  r[1])).sum())

1.4992800000000002


1.50754


1.5158


1.5009100000000002


In [8]:
size = len(results[0][1])
pd.Series(map(costs.get,  int(size/2)*['Light Quarantine'] + int(size/2)*['Social Distancing'])).sum()/2

0.6577000000000001

In [11]:
from simulate_pandemic import main

Loading Graph...Done!


In [12]:
data, contacts_infected, pop_matrix =  main()

  0%|▎                                                                                 | 2/499 [00:00<00:29, 16.96it/s]

{'work': 0, 'school': 0, 'home': 0, 'neighbor': 0}


100%|████████████████████████████████████████████████████████████████████████████████| 499/499 [01:15<00:00,  6.64it/s]


In [13]:
df = make_SIR_graph(data)